# Análise Comparativa

#### **Importando Bibliotecas**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, ShuffleSplit, KFold, cross_validate

#### **Leitura dos Dados**

In [2]:
df= pd.read_csv("../data/raw/wines_SPA.csv")
df.head()

,winery,wine,year,rating,num_reviews,country,region,price,type,body,acidity
0,Teso La Monja,Tinto,2013,4.9,58,Espana,Toro,995.00,Toro Red,5.0,3.0
1,Artadi,Vina El Pison,2018,4.9,31,Espana,Vino de Espana,313.50,Tempranillo,4.0,2.0
2,Vega Sicilia,Unico,2009,4.8,1793,Espana,Ribera del Duero,324.95,Ribera Del Duero Red,5.0,3.0
3,Vega Sicilia,Unico,1999,4.8,1705,Espana,Ribera del Duero,692.96,Ribera Del Duero Red,5.0,3.0
4,Vega Sicilia,Unico,1996,4.8,1309,Espana,Ribera del Duero,778.06,Ribera Del Duero Red,5.0,3.0


In [3]:
df_dict = pd.read_csv('../data/external/dictionary.csv')
df_dict

,variavel,descricao,tipo,subtipo
0,winery,Nome da vinícola,qualitativa,nominal
1,wine,Nome do vinho,qualitativa,nominal
2,year,Ano em que as uvas foram colhidas,quantitativa,discreta
3,rating,Avaliação média dada ao vinho pelos usuários [...,quantitativa,contínua
4,num_reviews,Número de usuários que avaliaram o vinho,quantitativa,discreta
5,country,País de origem [Espanha],inútil,inútil
6,region,Região do vinho,qualitativa,nominal
7,price,Preço em euros [€],quantitativa,contínua
8,type,Variedade de vinho,qualitativa,nominal
9,body,"Pontuação de corpo, definida como a riqueza e ...",qualitativa,ordinal


#### **Renomeando as colunas**

In [4]:
mapeamento_colunas = {
    'winery': 'vinícola',
    'wine': 'vinho',
    'year': 'ano',
    'rating': 'avaliação',
    'num_reviews': 'num_avaliações',
    'country': 'país',
    'region': 'região',
    'price': 'preço',
    'type': 'tipo',
    'body': 'corpo',
    'acidity': 'acidez'
}

In [5]:
def renomear_colunas(df, mapping):
    df.rename(columns=mapping, inplace=True)
    return df

In [6]:
df_dict['variavel'] = df_dict['variavel'].replace(mapeamento_colunas)
df = renomear_colunas(df, mapeamento_colunas)

## Tratamento e Transformação de dados

#### **Verificando os Valores Faltantes e Dados Duplicados**

In [7]:
print(df.duplicated().sum())

5452


In [8]:
df = df.drop(df[df.duplicated()].index, axis=0)
df.reset_index()
print(f'Número de Duplicados: {df.duplicated().sum()}')

Número de Duplicados: 0


In [9]:
df['ano'] = df['ano'].replace('N.V.', np.nan)
df['ano'] = df['ano'].astype(pd.Int64Dtype())

In [10]:
print(f'Verificando valores faltantes : \n\n{df.isna().sum()}')

Verificando valores faltantes : 

vinícola            0
vinho               0
ano                72
avaliação           0
num_avaliações      0
país                0
região              0
preço               0
tipo              106
corpo             271
acidez            271
dtype: int64


#### **Verificando os Valores Discrepantes**

In [11]:
# Função para identificar outliers usando o IQR
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)  # Primeiro quartil
    Q3 = df[column].quantile(0.75)  # Terceiro quartil
    IQR = Q3 - Q1                     # Intervalo Interquartil
    
    lower_bound = Q1 - 1.5 * IQR      # Limite inferior
    upper_bound = Q3 + 1.5 * IQR      # Limite superior
    
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

In [12]:
# Detectando outliers apenas para colunas numéricas
outliers_summary = []

for col in df.select_dtypes(include=[np.number]).columns:  # Filtra apenas colunas numéricas
    outliers = detect_outliers_iqr(df, col)
    if not outliers.empty:
        outliers_summary.append({'Coluna': col, 'Quantidade de Outliers': len(outliers)})

outliers_df = pd.DataFrame(outliers_summary)

print(outliers_df)

           Coluna  Quantidade de Outliers
0             ano                     165
1       avaliação                       2
2  num_avaliações                     242
3           preço                     245
4           corpo                      34
5          acidez                     105


In [13]:
df['preço'] = np.log1p(df['preço'])

# Pré-Processamento dos Dados

#### **Definição das Variáveis do Modelo de Previsão**

In [14]:
# Variável alvo
target_variable = 'preço'

# Variáveis inúteis
useless_variables = (
    df_dict
    .query("tipo == 'inútil'")
    .variavel
    .to_list()
)

# Variáveis nominais
nominal_variables = (
    df_dict
    .query("subtipo == 'nominal' and variavel != @target_variable")
    .variavel
    .to_list()
)

# Variáveis ordinais
ordinal_variables = (
    df_dict
    .query("subtipo == 'ordinal' and variavel != @target_variable")
    .variavel
    .to_list()
)

# Variáveis contínuas
continuous_variables = (
    df_dict
    .query("subtipo == 'contínua' and variavel != @target_variable")
    .variavel
    .to_list()
)

# Variáveis discretas
discrete_variables = (
    df_dict
    .query("subtipo == 'discreta' and variavel != @target_variable")
    .variavel
    .to_list()
)

In [15]:
X = df.drop(columns=[target_variable] + useless_variables)
y = df[target_variable]

In [16]:
# Exibindo as categorias de variáveis
print("Nominal Variables:", nominal_variables)
print("Ordinal Variables:", ordinal_variables)
print("Continuous Variables:", continuous_variables)
print("Discrete Variables:", discrete_variables)

Nominal Variables: ['vinícola', 'vinho', 'região', 'tipo']
Ordinal Variables: ['corpo', 'acidez']
Continuous Variables: ['avaliação']
Discrete Variables: ['ano', 'num_avaliações']


#### **Pré-processamento das Variáveis: Imputação e Codificação**

In [17]:
# Nominal variables: imputação de valores faltantes e codificação one-hot
nominal_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy="most_frequent")),  # Preenchimento com o valor mais frequente
    ("encoding", OneHotEncoder(sparse_output=False, handle_unknown="infrequent_if_exist"))  # Codificação One-Hot
])

# Ordinal variables: imputação de valores faltantes e codificação ordinal
ordinal_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy="median")),  # Preenchimento com a mediana
    ("encoding", OrdinalEncoder())  # Codificação ordinal
])

# Continuous variables: imputação de valores faltantes e normalização
continuous_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy="mean")),  # Preenchimento com a média
    ("normalization", StandardScaler())  # Normalização
])

# Discrete variables: imputação de valores faltantes e normalização
discrete_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy="median")),  # Preenchimento baseado em K vizinhos mais próximos
    ("normalization", StandardScaler())  # Normalização
])

In [18]:
# Criação do ColumnTransformer para processar cada tipo de variável
preprocessor = ColumnTransformer([
    ("nominal", nominal_preprocessor, nominal_variables),   
    ("ordinal", ordinal_preprocessor, ordinal_variables),   
    ("continuous", continuous_preprocessor, continuous_variables),  
    ("discrete", discrete_preprocessor, discrete_variables), 
])

In [19]:
preprocessor

ColumnTransformer(transformers=[('nominal',
                                 Pipeline(steps=[('missing',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoding',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                sparse_output=False))]),
                                 ['vinícola', 'vinho', 'região', 'tipo']),
                                ('ordinal',
                                 Pipeline(steps=[('missing',
                                                  SimpleImputer(strategy='median')),
                                                 ('encoding',
                                                  OrdinalEncoder())]),
                                 ['corpo', 'acidez']),
                                ('continuous',
                                 Pipeline(steps=[('missing', SimpleImputer()),
                                                 ('normalization',
                                                  StandardScaler())]),
                                 ['avaliação']),
                                ('discrete',
                                 Pipeline(steps=[('missing',
                                                  SimpleImputer(strategy='median')),
                                                 ('normalization',
                                                  StandardScaler())]),
                                 ['ano', 'num_avaliações'])])

# **Avaliação e Comparação de Modelos**

#### **Validação Cruzada e Métricas de Desempenho**

In [20]:
from sklearn.ensemble import RandomForestRegressor

In [21]:
modelos = [DummyRegressor(strategy="mean"), LinearRegression(), SVR(), RandomForestRegressor(n_estimators=100, random_state=42)]

In [22]:
metricas = [
    'neg_mean_absolute_error',
    'neg_mean_squared_error',
    'neg_mean_absolute_percentage_error',
    'r2',
]

monte_carlo = ShuffleSplit(n_splits=10, test_size=.2, random_state=42)

In [23]:
results_total = None
for modelo in modelos:
    model_name = modelo.__class__.__name__
    print(f"rodando para o modelo: {model_name}")
    approach = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('modelo', modelo),
    ])

    scores = cross_validate(
        approach, X, y,
        scoring=metricas, 
        cv=monte_carlo
    )
    results_model = pd.DataFrame(scores)
    results_model['Modelo'] = model_name
    if results_total is None:
        results_total = results_model
    else:
        results_total = pd.concat([results_total, results_model])

rodando para o modelo: DummyRegressor
rodando para o modelo: LinearRegression
rodando para o modelo: SVR
rodando para o modelo: RandomForestRegressor


In [24]:
results_total.groupby('Modelo').agg(['mean', 'std']).T

Modelo                                        DummyRegressor  \
fit_time                                mean        0.058314   
                                        std         0.014035   
score_time                              mean        0.020805   
                                        std         0.004869   
test_neg_mean_absolute_error            mean       -0.792764   
                                        std         0.027047   
test_neg_mean_squared_error             mean       -1.054131   
                                        std         0.077255   
test_neg_mean_absolute_percentage_error mean       -0.193789   
                                        std         0.008106   
test_r2                                 mean       -0.001687   
                                        std         0.002000   

Modelo                                        LinearRegression  \
fit_time                                mean      1.116493e+00   
                                        std       2.576957e-01   
score_time                              mean      1.975935e-02   
                                        std       4.280900e-03   
test_neg_mean_absolute_error            mean     -8.784713e+10   
                                        std       9.317223e+10   
test_neg_mean_squared_error             mean     -1.870039e+23   
                                        std       3.372419e+23   
test_neg_mean_absolute_percentage_error mean     -2.487467e+10   
                                        std       2.584747e+10   
test_r2                                 mean     -1.723517e+23   
                                        std       3.123433e+23   

Modelo                                        RandomForestRegressor       SVR  
fit_time                                mean              10.007238  1.819722  
                                        std                2.627242  0.343385  
score_time                              mean               0.069073  0.672386  
                                        std                0.046093  0.173253  
test_neg_mean_absolute_error            mean              -0.374220 -0.336210  
                                        std                0.013652  0.010954  
test_neg_mean_squared_error             mean              -0.258897 -0.213778  
                                        std                0.021582  0.017288  
test_neg_mean_absolute_percentage_error mean              -0.091684 -0.082441  
                                        std                0.003643  0.003075  
test_r2                                 mean               0.753764  0.796460  
                                        std                0.014106  0.014804

* **Predição do Modelo**

Divisão de treino e teste:

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
# Aplicar o pré-processador aos dados de treino e teste
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)


In [27]:
# Verificando o tamanho dos dados transformados
print("Shape of X_train_transformed:", X_train_transformed.shape)
print("Shape of X_test_transformed:", X_test_transformed.shape)

Shape of X_train_transformed: (1638, 1268)
Shape of X_test_transformed: (410, 1268)


In [28]:
# Dicionário para armazenar os modelos treinados
trained_models = {}
# Treinando os modelos com os dados transformados
for model in modelos:
    model_name = type(model).__name__
    model.fit(X_train_transformed, y_train)
    trained_models[model_name] = model

In [29]:
# Fazendo predições
predictions = {}
for model_name, trained_model in trained_models.items():
    y_pred = trained_model.predict(X_test_transformed)
    predictions[model_name] = y_pred
    print(f"Predictions for {model_name}:")
    print(y_pred[:5])  # Exibindo as primeiras 5 predições


Predictions for DummyRegressor:
[4.17956997 4.17956997 4.17956997 4.17956997 4.17956997]
Predictions for LinearRegression:
[4.73601734e+11 4.05755615e+00 7.41448975e+00 4.62103271e+00
 4.07049561e+00]
Predictions for SVR:
[3.42159783 3.78997048 7.22446071 4.50556286 3.40738925]
Predictions for RandomForestRegressor:
[3.72484582 3.96430207 7.02458752 4.53668817 3.49149499]
